# Первая БД

In [1]:
import sqlite3 as sl

In [2]:
authors = sl.connect("authors_database.db")

In [3]:
cursor = authors.cursor()

In [4]:
cursor.execute('''
CREATE TABLE author (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    login TEXT NOT NULL,
    email TEXT NOT NULL
);
''')

In [5]:
cursor.execute('''
CREATE TABLE blog (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    owner_id INTEGER NOT NULL,
    name TEXT NOT NULL,
    description TEXT,
    FOREIGN KEY (owner_id) REFERENCES author(id)
);
''')

In [6]:
cursor.execute('''
CREATE TABLE post (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    header TEXT NOT NULL,
    text TEXT NOT NULL,
    author_id INTEGER NOT NULL,
    blog_id INTEGER NOT NULL,
    FOREIGN KEY (author_id) REFERENCES author(id),
    FOREIGN KEY (blog_id) REFERENCES blog(id)
);
''')

In [59]:
# По заданию нужны сообщения, пришлось собрать еще одну табличку, связанную с постом
cursor.execute('''
CREATE TABLE message (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text TEXT NOT NULL,
    author_id INTEGER NOT NULL,
    post_id INTEGER NOT NULL,
    FOREIGN KEY (author_id) REFERENCES author(id),
    FOREIGN KEY (post_id) REFERENCES post(id)
);
''')

ProgrammingError: Cannot operate on a closed database.

In [7]:
authors.commit()
authors.close()

# Делаем вторую бд

In [8]:
logs = sl.connect("logs_database.db")

In [9]:
cursor = logs.cursor()

In [10]:
cursor.execute('''
CREATE TABLE space_type (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);
''')

In [11]:
cursor.execute('''
INSERT INTO space_type (name) VALUES ('global'), ('blog'), ('post');
''')

In [12]:
cursor.execute('''
CREATE TABLE event_type (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);
''')

In [13]:
cursor.execute('''
INSERT INTO event_type (name) VALUES ('login'), ('comment'), ('create_post'), ('delete_post'), ('logout');
''')

In [14]:
cursor.execute('''
CREATE TABLE logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    datetime DATETIME NOT NULL,
    user_id INTEGER NOT NULL,
    space_type_id INTEGER NOT NULL,
    event_type_id INTEGER NOT NULL,
    FOREIGN KEY (space_type_id) REFERENCES space_type(id),
    FOREIGN KEY (event_type_id) REFERENCES event_type(id)
);
''')

In [15]:
logs.commit()
logs.close()

# Заполним первую базу данных с пользователями, блогами и постами

In [16]:
authors = sl.connect("authors_database.db")
cursor = authors.cursor()

In [17]:
for i in range(10):
    cursor.execute(f'''
    INSERT INTO author (login, email) VALUES ('test_user{i}', 'test{i}@example.com');
    ''')

In [18]:
for i in range(10):
    cursor.execute(f'''
    INSERT INTO blog (owner_id, name, description) VALUES (
        (SELECT id from author where login='test_user{i}'),
        'Blog test {i}',
        'Lorem ipsum, i love pancakes and i love my mother, therefore my mother is a pancake'
    );
    ''')

In [19]:
cursor.execute('''
INSERT INTO post (header, text, author_id, blog_id) VALUES (
    'First post',
    'First text post lol',
    (SELECT id from author where login='test_user'),
    (SELECT id from blog where name='Blog test 1')

);
''')

In [20]:
authors.commit()
authors.close()

# Заполним вторую базу данных

In [21]:
logs = sl.connect("logs_database.db")
cursor = logs.cursor()

In [22]:
cursor.execute('''
INSERT INTO logs (datetime, user_id, space_type_id, event_type_id) VALUES (
    '2025-03-01 15:00:00',
    1,
    (SELECT id FROM space_type WHERE name = 'global'),
    (SELECT id FROM event_type WHERE name = 'login')
);
''')

In [23]:
cursor.execute('''
INSERT INTO logs (datetime, user_id, space_type_id, event_type_id) VALUES (
    '2025-03-01 15:00:01',
    1,
    (SELECT id FROM space_type WHERE name = 'global'),
    (SELECT id FROM event_type WHERE name = 'logout')
);
''')

In [24]:
logs.commit()
logs.close()

# Заполним базы данных

In [49]:
from datetime import datetime as dt
import random

authors = sl.connect("authors_database.db")
auth = authors.cursor()

logs = sl.connect("logs_database.db")
log = logs.cursor()

def getDatetimeInRightWay():
    return dt.now().strftime("%Y-%m-%d %H:%M:%S")

def isLoginInBase(login: str):
    auth.execute(f'''
    SELECT * FROM author WHERE login = ?
    ''', (login,))
    return auth.fetchall()

def setLoginInBase(login: str, email: str = ""):
    if not email: email = f"{login}@mail.ru"
    auth.execute('''
    INSERT INTO author (login, email) VALUES (?, ?);
    ''', (login, email))
    return authors.commit()



def createBlog(login: str, name: str, description: str):
    if not isLoginInBase(login):
        setLoginInBase(login)
    auth.execute('''
    INSERT INTO blog (owner_id, name, description) VALUES (
        (SELECT id from author where login = ?),
        ?,
        ?
    );
    ''', (login, name, description))
    
    return authors.commit()


# Вот тут я немного не понял, вроде как были только посты, блог и пользователи, откуда взялись комменты - вопрос интересный.
def createPost(login: str, header: str, test: str, blog_id: int):
    authors.commit('''
    INSERT INTO post (header, text, author_id, blog_id) VALUES (
        ?,
        ?,
        (SELECT id from author where login = ?),
        ?
    );
    ''', (headr, text, login, blog_id))
    


NameError: name 'q' is not defined

In [58]:
\

[(1, 'test_user', 'test@example.com')]